In [6]:
import pandas as pd
import torch

from tqdm.notebook import tqdm
from transformers import AutoModelForSequenceClassification, BertTokenizerFast
from transformers import pipeline

from ydata_profiling import ProfileReport

In [3]:
import warnings

warnings.filterwarnings('ignore')

# Сентимент анализ

# --------------------------------------------------------------------------------------------------------------------------------------


In [2]:
articles = pd.read_pickle('data/processed_articles.pkl')
comments = pd.read_pickle('data/processed_comments.pkl')

In [3]:
LABELS = ('NEUTRAL', 'POSITIVE', 'NEGATIVE')

In [5]:
tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True)


@torch.no_grad()
def predict(tokens: list[str]) -> str:
    text = ' '.join(tokens)
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).numpy()
    return LABELS[predicted.item()]


In [6]:
tqdm.pandas(desc='sentiment')
comments['sentiment'] = comments['tokens'].progress_apply(predict)

sentiment:   0%|          | 0/18750 [00:00<?, ?it/s]

In [52]:
comments['sentiment'] = comments['sentiment'].str.lower()

In [109]:
text, tokens, sentiment = comments.sample(1)[['text', 'tokens', 'sentiment']].iloc[0]

print(' '.join(tokens))
print('-' * 40)
print(sentiment)

ну вот собственно из за тот что мы вместо чтобы изучать и развивать прогрессивный технология минусовать каждый они упоминание мы и получать тормозить скайп
----------------------------------------
NEGATIVE


In [112]:
articles.to_pickle('data/processed_articles.pkl')
comments.to_pickle('data/processed_comments.pkl')

# Фильтрация

# --------------------------------------------------------------------------------------------------------------------------------------


In [110]:
filtered_comments = comments[~comments['tokens'].apply(lambda x: len(x) == 0)]
filtered_comments = filtered_comments[~filtered_comments['purged_tokens'].apply(lambda x: len(x) == 0)]

print(f"comments: {comments.shape[0]} -> {filtered_comments.shape[0]}")
print(f"diff - {comments.shape[0] - filtered_comments.shape[0]}")

comments: 18750 -> 18542
diff - 208


In [111]:
comments = filtered_comments

In [112]:
articles.to_pickle('data/processed_articles.pkl')
comments.to_pickle('data/processed_comments.pkl')

# Анализ эмоций

# --------------------------------------------------------------------------------------------------------------------------------------


In [42]:
articles = pd.read_pickle('data/processed_articles.pkl')
comments = pd.read_pickle('data/processed_comments.pkl')

In [43]:
model = pipeline(model="seara/rubert-tiny2-ru-go-emotions")

def predict(tokens: list[str]):
    ' '.join(tokens)
    result = model(' '.join(tokens))[0]
    return result['label']

In [44]:
tqdm.pandas(desc='emotion prediction')
comments['emotion'] = comments['tokens'].progress_apply(predict)

emotion prediction:   0%|          | 0/18542 [00:00<?, ?it/s]

In [48]:
tokens, emotion = comments.sample(1)[['tokens', 'emotion']].iloc[0]

print(' '.join(tokens))
print('-' * 40)
print(emotion)

позволить себя не согласиться не согласиться с чем конкретно вы не привести ни один пример который бы противоречить тот что я сказать
----------------------------------------
disapproval


In [54]:
articles.to_pickle('data/processed_articles.pkl')
comments.to_pickle('data/processed_comments.pkl')

# Профайлинг

# --------------------------------------------------------------------------------------------------------------------------------------


In [55]:
articles = pd.read_pickle('data/processed_articles.pkl')
comments = pd.read_pickle('data/processed_comments.pkl')

In [56]:
profiled_comments = comments.drop(['text', 'article_id'], axis=1)

profiled_comments['tokens'] = profiled_comments['tokens'].apply(' '.join)
profiled_comments['purged_tokens'] = profiled_comments['purged_tokens'].apply(' '.join)

In [57]:
profiled_comments.dtypes

author                             object
published_datetime    datetime64[ns, UTC]
votes                               int64
tokens                             object
purged_tokens                      object
sentiment                          object
emotion                            object
dtype: object

In [58]:
profiled_comments.sample(10)

,author,published_datetime,votes,tokens,purged_tokens,sentiment,emotion
5165,SkywardFire,2023-12-04 21:07:15+00:00,0,кстати осознанно писать на не сам новый версия...,кстати осознанно версия интерпретатор причина,neutral,neutral
12097,maxkain,2023-05-24 11:12:03+00:00,1,высокий математик тоже сложный для понимание з...,высокий математик сложный понимание ирония отн...,neutral,disapproval
6851,gmtd,2023-10-03 02:30:57+00:00,0,начать с тот что этот статья быть написать на ...,статья написать основа собеседование смочь фре...,negative,confusion
2475,krabdb,2024-01-20 12:43:18+00:00,0,кстати с март российский банк запретить переда...,кстати март российский банк запретить передава...,neutral,neutral
64,scarab,2024-03-12 12:12:59+00:00,0,сколько разработчик при это быть в команда про...,разработчик команда проект секрет,neutral,confusion
4363,Neusser,2023-12-06 06:20:24+00:00,0,использование ноутбук не ограничиваться интерф...,использование ноутбук ограничиваться интерфейс...,neutral,neutral
11022,inkelyad,2023-07-03 14:26:48+00:00,0,криптомикросхема hsm стоять американский деньг...,криптомикросхема hsm американский деньга штука...,neutral,neutral
13820,Voldar,2023-03-16 06:39:52+00:00,0,очень много весь происходить и не хватать сила...,происходить хватать записывать план продолжить,neutral,optimism
18011,demimurych,2022-10-19 03:44:18+00:00,0,в то время веб заниматься программист а не тот...,веб заниматься программист изучать javascript ...,neutral,annoyance
8472,username1234512345,2023-09-11 12:39:00+00:00,0,ты про пипмайя вроде писать у себя что предлаг...,пипмайя вроде предлагать готовый движок сайт з...,negative,curiosity


In [59]:
report = ProfileReport(profiled_comments, title='Comments Report', explorative=True)
report.to_file("results/comments_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [60]:
profiled_articles = articles.drop(['id', 'url'], axis=1)

profiled_articles['tags'] = profiled_articles['tags'].apply(' '.join)

In [61]:
profiled_articles.dtypes

author                             object
published_datetime    datetime64[ns, UTC]
title                              object
complexity                         object
reading_time                        int64
views                               int64
tags                               object
votes                               int64
bookmarks                           int64
comments                            int64
dtype: object

In [62]:
profiled_articles.sample(10)

,author,published_datetime,title,complexity,reading_time,views,tags,votes,bookmarks,comments
378,alexandrareznikova11,2023-09-29 09:15:14+00:00,"Геймификация в охране труда: кому, зачем и как?",NaN,4,1300,Веб-разработка IT-инфраструктура Разработка иг...,2,12,1
751,thepythonicway,2023-02-21 14:21:55+00:00,Как изучать язык Python для веб-разработки в 2...,Простой,17,27000,Веб-разработка Python Django API Карьера в IT-...,2,223,7
717,RostislavDugin,2023-03-11 15:12:08+00:00,Стартап в Соло. Часть 3: упрощаем продукт,Простой,5,3100,Веб-разработка Развитие стартапа,3,32,0
921,Vordgi,2022-10-24 11:30:02+00:00,Next.js v13. Что нового и чего ждать в будущем,NaN,6,6600,Веб-разработка JavaScript ReactJS Поисковая оп...,2,16,1
213,cherkalexander,2023-12-18 07:02:23+00:00,Как я документацию для дизайн-токенов в Storyb...,Простой,10,2600,Веб-разработка Open source ReactJS Дизайн,9,26,0
406,m5xim,2023-09-18 06:03:02+00:00,Подключение Twig шаблонов в WordPress: Как изб...,Средний,4,880,WordPress Веб-разработка PHP HTML,0,10,0
942,Anastasia_rova,2022-10-10 15:53:27+00:00,Разбираемся в анимациях и временных переходах ...,NaN,8,38000,Блог компании Нетология Веб-дизайн Веб-разрабо...,1,90,0
785,aio350,2023-01-31 08:01:01+00:00,JavaScript: заметка об операторе конвейера,NaN,8,3700,Блог компании Timeweb Cloud Веб-разработка Jav...,15,24,1
520,fat32elena,2023-07-18 07:01:45+00:00,"Если ваш фронтендер перестал бояться IE6, пока...",NaN,7,22000,Блог компании Яндекс Веб-разработка CSS JavaSc...,64,75,85
817,StiPAFk,2023-01-09 14:20:57+00:00,Управление состоянием в React приложениях,NaN,3,6500,Веб-разработка JavaScript ReactJS TypeScript,1,46,39


In [63]:
report = ProfileReport(profiled_articles, title='Comments Report', explorative=True)
report.to_file("results/articles_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# --------------------------------------------------------------------------------------------------------------------------------------
